In [1]:
import numpy as np
import pandas as pd
import sklearn
import keras

Using TensorFlow backend.


In [160]:
movies = pd.read_csv('archive/movies_metadata.csv')

In [161]:
movies = movies.filter(['original_title', 'id', 'imdb_id','vote_average'])

In [162]:
movies["reviews"] = [[] for _ in range(len(movies))]
movies.head()

,original_title,id,imdb_id,vote_average,reviews
0,Toy Story,862,tt0114709,7.7,[]
1,Jumanji,8844,tt0113497,6.9,[]
2,Grumpier Old Men,15602,tt0113228,6.5,[]
3,Waiting to Exhale,31357,tt0114885,6.1,[]
4,Father of the Bride Part II,11862,tt0113041,5.7,[]


In [165]:
import json
import os
import re
import requests

REVIEW_PATTERN = 'https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={key}'
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()


def get_tmdb_reviews():
    """ request reviews with 'id' from TMDB and append them to the reviews column """
    k=0
    for index, row in movies.iterrows():
        id = row['id']
        r = _get_json(REVIEW_PATTERN.format(key=KEY,movie_id=id))
        
        if not r:
            print("request cannot be located")
            continue
        
        n = r.get('total_results')
        
        reviews = []
        for i in range(n):
            k += 1
            result = r['results'][i]
            review = result.get('content')
            reviews.append(review)
            #rating = result.get('author_details').get('rating')
            
        movies.at[index, 'reviews'] = reviews
        if not reviews:
            movies.at[index, 'reviews'] = float("NaN")
        
        # SET LIMIT
        if index > 20:
            print(k)
            break

    return None

get_tmdb_reviews()
movies.head()

13


,original_title,id,imdb_id,vote_average,reviews
0,Toy Story,862,tt0114709,7.7,[This movie came out when I was three. Now I'm...
1,Jumanji,8844,tt0113497,6.9,"[Throw the dice and take a turn, Jumanji made ..."
2,Grumpier Old Men,15602,tt0113228,6.5,NaN
3,Waiting to Exhale,31357,tt0114885,6.1,NaN
4,Father of the Bride Part II,11862,tt0113041,5.7,NaN


In [164]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

import string

def tokenize(list):
    t = []
    
    for s in list:
        tokens = s.split()

        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        
        tokens = [word for word in tokens if word.isalpha()]
        
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]
        
        tokens = [word for word in tokens if len(word) > 1]
        
        t.append(tokens)
    
    t = [word.lower() for sl in t for word in sl]
        
    lemmatizer = WordNetLemmatizer()
    for w in t:
        w = lemmatizer.lemmatize(w)
    
    # returns a flat list of strings
    return t

def clean_data():
    for index, row in movies.iterrows():
        review = row['reviews']
        movies.at[index, 'reviews'] = tokenize(review)
        
        # SET LIMIT
        if index > 20:
            print(k)
            break
            
    return None

clean_data()
df.dropna(subset = ["reviews"], inplace=True)
movies.head(20)

[nltk_data] Downloading package stopwords to C:\Users\Gina
[nltk_data]     Wu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: 'float' object is not iterable

In [ ]:
def encode_text():
    
    return padded

In [138]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

def custom_model(kength, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model